# Getting Started

Mondocks is a simple library with one purpose, create MongoDB Json Commands these commands can be executed by the various mongodb drivers that exist out there or even by the mongo shell.

You can get started by typing on your console `dotnet add package Mondocks` and you'll be able to open the namespaces and modules that are included in this library

## The Basics

Mondocks follows this convention 
```sh
"command name" "collection name" {
    "query contents"
    "query contents"
    "query contents"
    "query contents"
}
```
So most of the commands will look the same, they will just have different options depending on what do you need to do with them.
the *builder*'s signatures look a bit crazy and full of generics that might be confusing, in reallity we add the generics so you can use any kind of type you neeed to use, if you want to do query filters by Records because you have a shared filters project, you can query by anonymous records as if you were using a *javascript* object as well it's your choice we try to stay as away as posible to let you decide how do you want to handle your data types

Let's start with some CRUD examples

# Insert (Create)

Inserts are fairly straight forward you just need to suply a sequence of objects they can be either records or anonymous records, if you want to go a little bit more dynamix you can also pass boxed objects that have different properties.



In [1]:
#!fsharp
#r "nuget: Mondocks"

open Mondocks.Queries

(* let's start by doing some Create operations 
   we'll try to show how can do mongo like constructs
   and then you can add some type safety on top of that
*)

let createSinglePost (title: string) (content: string) (author: string) = 
    insert "posts" {
        documents
            [
                /// here we use anonymous records as if they were *mongo* (or *javascript*) objects
                {| title = title; content = content;  author = author |}
            ]
    }
let insertquery = createSinglePost "This is my first post" "This is the content" "Angel D. Munoz"
printfn $"%s{insertquery}"


Installed package Mondocks version 0.2.0

{"insert":"posts","documents":[{"author":"Angel D. Munoz","content":"This is the content","title":"This is my first post"}]}

In [1]:
#!fsharp
(* Now, what if you want to insert multiple values?
   well creating a query for each is not efficient so you just need to
   pass an IEnumerable, e.g. a list, an array or a sequence with the data you want to insert
   but this makes things a litte bit weird so why don't we make a type for it?
*)
type InsertPost = {| title: string; content: string;  author: string |}

let createMultiplePosts (posts: InsertPost list) =
    insert "posts" { documents posts }

let postsToInsert =
    createMultiplePosts [ {| title = "This is my first post"; content =  "This is the content"; author = "Angel D. Munoz" |}
                          {| title = "This is my second post"; content =  "This is the content"; author = "Angel D. Munoz" |}
                          {| title = "This is my third post"; content =  "This is the content"; author = "Angel D. Munoz" |}
                          {| title = "This is my fourth post"; content =  "This is the content"; author = "Angel D. Munoz" |} ]

// let's compare both
printfn $"%s{insertquery}\n %s{postsToInsert}"


{"insert":"posts","documents":[{"author":"Angel D. Munoz","content":"This is the content","title":"This is my first post"}]}

{"insert":"posts","documents":[{"author":"Angel D. Munoz","content":"This is the content","title":"This is my first post"},{"author":"Angel D. Munoz","content":"This is the content","title":"This is my second post"},{"author":"Angel D. Munoz","content":"This is the content","title":"This is my third post"},{"author":"Angel D. Munoz","content":"This is the content","title":"This is my fourth post"}]}

In [1]:
#!fsharp
(* Let's go very very dynamic in this example let's add objects that
   have different kinds of properties, this might not be un-common if your database
   is being filled with data coming from other languages like javascript or python
*)

let createMultipleDynamicPosts = 
    insert "posts" {
        documents [
            box {| title = "This is my first post"; content =  "This is the content"; author = "Angel D. Munoz" |}
            box {| title = "This is my second post"; content =  "This is the content"; author = "Mike Thompson"; createdAt = DateTime.Now |}
            box {| title = "This is my third post"; content =  "This is the content"; author = "Frank Miller"; tags = [ "mongodb"; "fsharp"; "nosql"] |}
        ]
    }

printfn $"%s{createMultipleDynamicPosts}"

{"insert":"posts","documents":[{"author":"Angel D. Munoz","content":"This is the content","title":"This is my first post"},{"author":"Mike Thompson","content":"This is the content","createdAt":{"$date":{"$numberLong":"1608146332584"}},"title":"This is my second post"},{"author":"Frank Miller","content":"This is the content","tags":["mongodb","fsharp","nosql"],"title":"This is my third post"}]}

> Now I don't think I have to mention it but... that kind of dynamic data can be a little bit *problematic* if you try to use it from F# but business have different requirements all of the time as we mentioned earlier, we want to be as flexible as you need it to be as long as it fits mongo's way of doing things

If you inspect a little the last output you'll see something that might not look like what you intended. Let's take a look for example at the `createdAt` property in the second post
`"createdAt":{"$date":{"$numberLong":"1608135821957"}` this is what's called the [MongoDB](https://docs.mongodb.com/manual/reference/mongodb-extended-json/) extended json spec and it's basically a way to represent binary data (remember MongoDB uses BSON not JSON) in json with the least amount of data loss

# Find (Read)

Read operations can be really simple or they can be complex as well as with the insert queries, most of the time you should only need a couple of properties like `filter`, `limit`, `skip` and maybe `collation`.

Also remember that these filters must be done in the way you'd do them with mongo so you have to use `$in` `$or` and all of those operators, you can find that reference in the mongodb documentation about [Operators](https://docs.mongodb.com/manual/reference/operator/query/)

Let's get onto it.

In [1]:
#!fsharp
#r "nuget: Mondocks"

open Mondocks.Queries

(* First, let's do some 'raw' queries using anonymous records *)

let productsCostAbovePrice (price: float) =
    find "products" {
        filter {| price = {| ``$gt``= price |} |}
    }

printfn $"%s{productsCostAbovePrice 10.25 }"

{"find":"products","filter":{"price":{"$gt":{"$numberDouble":"10.25"}}}}

In [1]:
#!fsharp
(* You can also leverage the dynamic nature of the anonymous
   records to build filters on top of previous ones
*)

type GreaterThanFloat = {| ``$gt``: float |}
type EqualsToInt = {| ``$eq``: int |}
type PriceFilter = {| price: GreaterThanFloat |}
type BatchNumFilter = {|  batchNum: EqualsToInt |}


let greaterThanFloat (value: float): GreaterThanFloat = 
    {| ``$gt`` = value |}

let equalsToInt (value: int): EqualsToInt = 
    {| ``$eq`` = value |}

let getPriceFilter (gt: float -> GreaterThanFloat) (minPrice: float) =
    {| price = gt minPrice |}

let getBatchNumFilter (eq: int -> EqualsToInt) (batch: int) =
    {| batchNum = eq batch |}

// Let's combine the outputs from the previous functions to create a filter
// with both filters
let abovePriceFromBatch (minPrice: float) (batch: int) =
    let priceFilter = getPriceFilter greaterThanFloat minPrice
    {| priceFilter with batchNum = equalsToInt batch |}


let ``find products above 250.30`` = 
    find "products" {
        filter (getPriceFilter greaterThanFloat 250.30)
    }

let ``find products from batch 10`` = 
    find "products" {
        filter (getBatchNumFilter equalsToInt 10)
    }

let ``find products above 250.30 from batch 10`` =
    find "products" {
        filter (abovePriceFromBatch 250.30 10)
    }

printf $"{``find products above 250.30``}\n{``find products from batch 10``}\n{``find products above 250.30 from batch 10``}"

{"find":"products","filter":{"price":{"$gt":{"$numberDouble":"250.3"}}}}

{"find":"products","filter":{"batchNum":{"$eq":{"$numberInt":"10"}}}}

{"find":"products","filter":{"batchNum":{"$eq":{"$numberInt":"10"}},"price":{"$gt":{"$numberDouble":"250.3"}}}}

In [1]:
#!fsharp
(* To do the usual pagination it's fairly simple as well *)

let paginatedProductsPerBatch (batch: int) (page: int) (itemsPerPage: int) = 
    find "products" {
        filter (getBatchNumFilter equalsToInt batch)
        skip ((page - 1 ) * itemsPerPage)
        limit itemsPerPage
    }

printfn $"%s{paginatedProductsPerBatch 10 2 20}"


{"find":"products","filter":{"batchNum":{"$eq":{"$numberInt":"10"}}},"skip":{"$numberInt":"20"},"limit":{"$numberInt":"20"}}

If you want more type-safety around your queries you can use Records instead of anonymous ones, that'll allow you to (up to some degree) do some refactorings but please bear in mind that this only  affects your queries, not the existing underlaying data inside your mongo database.
Also, but you can still use records to *augment* your filters

In [1]:
#!fsharp
open MongoDB.Bson

type Question = { question: string; answer: string; }
type TitleFilter = { title: string }
type Form =
    { _id: ObjectId
      title: string;
      questions: Question list
      user: ObjectId }

let filterQuestionsIn (questions: string list) =
    let questions =
        questions
        |> List.map (fun question -> {| question = question |})
    {| ``$in`` =  questions |}

let filterTitle (title: string) =
    { title = title }

let findFormsWithQuestions (questions: string list) =
    find "forms" {
        filter (filterQuestionsIn questions)
    }

let findFormsWithTitle (title: string) = 
    find "forms" {
        filter (filterTitle title)
    }

let findFormsWithQuestionsAndTitle (questions: string list) (title: string) =
    let formFilter =
        let title = filterTitle title
        // create a new filter from an existing record
        {| title with 
            question = filterQuestionsIn questions |}
    find "forms" {
        filter formFilter
    }

let ``find forms with Sample as a title`` = 
    findFormsWithTitle "Sample"

let ``find forms with two questions`` = 
    findFormsWithQuestions ["What's your name?"; "Where do you live?"]
    
let ``find forms with two questions and Sample as a title`` = 
    findFormsWithQuestionsAndTitle ["What's your name?"; "Where do you live?"] "Sample"

printfn $"%s{``find forms with Sample as a title``}\n%s{``find forms with two questions``}\n%s{``find forms with two questions and Sample as a title``}"


{"find":"forms","filter":{"title":"Sample"}}

{"find":"forms","filter":{"$in":[{"question":"What\u0027s your name?"},{"question":"Where do you live?"}]}}

{"find":"forms","filter":{"question":{"$in":[{"question":"What\u0027s your name?"},{"question":"Where do you live?"}]},"title":"Sample"}}

# Update

Update operations are fairly straight forward as well although, for the update queries there's a helper type that can be used named `UpdateQuery<'Query, 'Update, 'Hint>`

let's dive into it

In [1]:
#!fsharp
#r "nuget: Mondocks"

open MongoDB.Bson
open Mondocks.Types
open Mondocks.Queries

type User = { _id: ObjectId; name: string; age: int; email: string; }

let updateUsersEmail(payload: User * string) =
    let (user, email) = payload
    let updatedUser = { user with email = email }
    update "users" {
        updates [
            { q = {| _id = user._id |}; 
              u = updatedUser; 
              upsert = None; multi = None; 
              collation = None; 
              arrayFilters = None; 
              hint = None }
        ]
    }

/// Now, let's say for example further down the road you need to add a property to all of your existing users
/// think of this as a "migration" and setting a default value so it doesn't conflict with
/// your existing F# records

type UserUpgrade = { _id: ObjectId; name: string; age: int; email: string; withNewProperty: bool }

let upgradeUsersSchema =
    update "users" {
        updates [
            { q = {||};
              u = {| ``$set`` = {| withNewProperty = true |} |}
              upsert = None
              multi = Some true
              collation = None
              arrayFilters = None
              hint = None }
        ]
    }

let user = {_id = ObjectId.GenerateNewId(); name = "Daniel"; age = 28; email= "some@email.com" }
let newEmail = "new@email.com"
printfn $"%s{updateUsersEmail (user, newEmail)}\n%s{upgradeUsersSchema}"


{"update":"users","updates":[{"q":{"_id":{"$oid":"5fda69f944739a88f3dadd84"}},"u":{"_id":{"$oid":"5fda69f944739a88f3dadd84"},"name":"Daniel","age":{"$numberInt":"28"},"email":"new@email.com"}}]}

{"update":"users","updates":[{"q":{},"u":{"$set":{"withNewProperty":true}},"multi":true}]}

### BEWARE
executing queries against `{||}` (mongo's `{}`) is going to affect all documents inside a collection (specially if `multi = Some true` is provided)

So be really careful when you update your database using empty objects 

# Delete

Deletes are very much like Updates there's also a helper type named `DeleteQuery<'Delete, 'Hint, 'Comment>`

Let's check some samples

In [1]:
#!fsharp
#r "nuget: Mondocks"

open Mondocks.Queries

let deleteUserWhereName  (name: string) =
    delete "users" {
        deletes [
            { q = {| name = name |}
              /// if you want to delete all documents that match the criteria use
              /// `limit = 0`
              limit = 1
              collation = None
              hint = None
              comment = None }
        ]
    }


> You can do multiple delete queries (with updates as well)
>
> ***NOTE***: Please note that you need to box the queries if they have different matching criteria

In [1]:
#!fsharp
let deleteAllUsersWhereName (name: string) =
    { q = box {| name = name |}
      limit = 0
      collation = None
      hint = None
      comment = None }

let delete10YOStaleUsers() =
    { q = 
        // sorry if my query is incorrect I just want to showcase multiple deletes 😅
        box {| lastLogin = {| ``$lte`` = DateTimeOffset.Now.Subtract(TimeSpan.FromDays(3650.)) |} |}
      limit = 0
      collation = None
      hint = None
      comment = None }

let deleteAllStaleUsersFromDate = 
    delete "users" {
        deletes [
            deleteAllUsersWhereName "Angel"
            delete10YOStaleUsers()
        ]
    }

printfn $"%s{deleteAllStaleUsersFromDate}"

{"delete":"users","deletes":[{"q":{"name":"Angel"},"limit":{"$numberInt":"0"}},{"q":{"lastLogin":{"$lte":{"$date":{"$numberLong":"1292790752079"}}}},"limit":{"$numberInt":"0"}}]}